In [3]:
import pandas as pd
from threading import Thread
pd.set_option('max_colwidth',500)
import multiprocessing

In [4]:
import sys
sys.path.append("../scripts/")
import clean_text
from importlib import reload
reload(clean_text)
clean_text.clean('@esto es un #tweet. Hola!!😀')

'@ esto es un # tweet. hola ! 😀'

In [12]:
tweets = pd.read_csv('../datasets/tweets_202007072142.csv.zip')

In [20]:
df = tweets.copy()
df.index = df.tweet_id.values
print(len(df))
print(df.columns)
df.head()

2474317
Index(['tweet_id', 'conversation_id', 'user_id', 'created_at', 'lang',
       'retweet_count', 'favorite_count', 'reply_count', 'quote_count',
       'is_reply', 'replied_id', 'replied_user_id', 'is_quote', 'quoted_id',
       'full_text', 'source', 'scrapped_at', 'user_mentions_count',
       'hashtags_count', 'urls_count', 'media_count', 'replied_original'],
      dtype='object')


,tweet_id,conversation_id,user_id,created_at,lang,retweet_count,favorite_count,reply_count,quote_count,is_reply,...,is_quote,quoted_id,full_text,source,scrapped_at,user_mentions_count,hashtags_count,urls_count,media_count,replied_original
1000000284486139904,1000000284486139904,1000000284486139904,2841506345,2018-05-25 13:07:01.0,es,23,124,5,1,0,...,0,NaN,Feliz día de la patria para todos los cordobes...,Twitter for Android,2020-07-07 11:42:04.0,0,0,0,3,NaN
1000000308003696640,1000000308003696640,1000000284486139904,2841506345,2018-05-25 13:07:07.0,es,24,83,12,0,1,...,0,NaN,#25deMayo Es un día en el que todos tenemos qu...,Twitter for Android,2020-07-07 11:42:04.0,0,1,0,2,1.0
1000000521158234114,1000000521158234114,1000000521158234114,54414081,2018-05-25 13:07:58.0,es,1287,3959,183,36,0,...,0,NaN,Feliz #DíaDeLaPatria.\nQuiero reconocer a todo...,Twitter for Android,2020-07-07 03:58:05.0,0,2,0,1,NaN
1000000589944639488,1000000589944639488,1000000589944639488,140065192,2018-05-25 13:08:14.0,es,4,20,0,0,0,...,0,NaN,(AHORA) Escuela de Robótica #Misiones. ​El vic...,Twitter Web Client,2020-07-07 14:37:01.0,0,1,0,1,NaN
1000000951195971584,1000000951195971584,1000000951195971584,35778170,2018-05-25 13:09:40.0,und,22,119,12,0,0,...,1,9.999982e+17,#EnVivo #Tedeum https://t.co/ERZoTL6VAJ,Twitter for Android,2020-07-07 03:59:28.0,0,2,1,0,NaN


In [33]:
print(len(df[(df.favorite_count > 5) & (df.lang == 'es')]))
print(len(df[df.lang=='es']))
df.sample(1)[['tweet_id', 'full_text']].values[0]

1634530
2318754


array([1098586196102602753,
       'Las fotografías seleccionadas a mejor imagen del año del World Press Photo: https://t.co/WzlSlubtkk '],
      dtype=object)

In [40]:
from joblib import Parallel, delayed

def paralell_func(values, func, frac=4):
    r = Parallel(n_jobs=8, backend='threading')(delayed(func)(i) for i in values)
    return r

def clean_fn(row):
    row[1] = clean_text.clean(row[1])
    return row

cleaned_texts = paralell_func(df.sample(5)[['tweet_id', 'full_text']].values, clean_fn)

In [47]:
list(zip(*cleaned_texts))

[(1268769852006727680,
  1269327743747084289,
  1277969247973421068,
  1279129712640622597,
  1194485053558054912),
 ('qué perdidos que están mamita URL',
  '@ maxi_andres_m @ giacodiego @ horaciorlarreta jajaja los reyes son los padres..',
  'cadetes de la escuela nacional de náutica embarcaron para cumplir sus prácticas profesionales. se re',
  'en cuba hay comunistas, en argentina hay un congreso con cientos de representantes populistas hambri',
  'fue una noche intensa en @ johnysabina, de debate y confrontación de las ideas. gracias a la diputad')]

In [51]:
frac = multiprocessing.cpu_count()
values = df[['tweet_id', 'full_text']].values
cleaned_texts = paralell_func(values, clean_fn, frac)

In [57]:
to_insert = list(zip(*cleaned_texts))
df.loc[to_insert[0], 'full_text'] = to_insert[1]

In [61]:
df.head(20)[['full_text']]

,full_text
1000000284486139904,"feliz día de la patria para todos los cordobeses. aquí en carlos paz tenemos un día espectacular, el"
1000000308003696640,"# 25demayo es un día en el que todos tenemos que tirar para el mismo lado, para lograr el progreso y"
1000000521158234114,feliz # díadelapatria. quiero reconocer a todos los que hoy están cuidando a los argentinos en cada r
1000000589944639488,"ahora escuela de robótica # misiones. el vicegobernador oscar herrera ahuad, en reunión con los di"
1000000951195971584,# envivo # tedeum URL
1000000957688811520,🇦 🇷 ¡ feliz día de la patria ! que este # 25demayo nos encuentre unidos para seguir construyendo una arge
1000001099514990592,"¡ feliz # 25demayo ! ahora, llega # demañana con @ prossioficial y equipo por am 810, fm 97.9 y en https"
1000002159289749505,"el hombre que vino a transformar la argentina, que nos llenó de sueños y convicciones. el hombre que"
1000002696584318977,"el modelo económico de macri está agotado hace décadas, repite una historia condenada al fracaso. t"
1000002915916972033,# nuevaplazasanmartín # 25demayo @ ciudaddemendoza # mendoza 🇦 🇷 💙 URL


In [63]:
# df.to_pickle('../datasets/tweets_cleaned.pkl')

In [7]:
df = pd.read_pickle('../datasets/tweets_cleaned.pkl')

In [31]:
len(df[(df.favorite_count > 50) & (df.lang == 'es')])

796626

In [32]:
corpus_to_save = df[(df.favorite_count > 50) & (df.lang == 'es')].full_text.values
with open("../datasets/tweets_corpus.txt", "w", encoding='utf-8') as text_file:
    text_file.write(
        "\n".join(corpus_to_save)
    )